In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy

2018-03-16 14:54:42,272 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
# Remove non-alphabets from text
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [4]:
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt',
    'LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt',
    'LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 
    'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt',
    'LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 
    'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,
    'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt',
    'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']


In [5]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))


# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    lemmatizer = WordNetLemmatizer()
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        
        # tokenize that string
        tokens =  word_tokenize(string)

        yield tokens
        f.close()

        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [6]:
# Create all the court specific dictionaries
T = texts()
dictionary_DCDRC = corpora.Dictionary(text for text in T.DCDRC())
dictionary_NCDRC = corpora.Dictionary(text for text in T.NCDRC())
dictionary_SCDRC = corpora.Dictionary(text for text in T.SCDRC())
dictionary_DelhiHC = corpora.Dictionary(text for text in T.DelhiHC())
dictionary_JharkhandHC = corpora.Dictionary(text for text in T.JharkhandHC())
dictionary_JodhpurHC = corpora.Dictionary(text for text in T.JodhpurHC())
dictionary_KolkataHC = corpora.Dictionary(text for text in T.KolkataHC())
dictionary_SupremeCourt = corpora.Dictionary(text for text in T.SupremeCourt())
# ## check out dictionaries
# dictionary_DCDRC.save_as_text('newdictionary_DCDRC.txt') 
# dictionary_NCDRC.save_as_text('newdictionary_NCDRC.txt') 
# dictionary_SCDRC.save_as_text('newdictionary_SCDRC.txt') 
# dictionary_DelhiHC.save_as_text('newdictionary_DelhiHC.txt') 
# dictionary_JharkhandHC.save_as_text('newdictionary_JharkhandHC.txt') 
# dictionary_JodhpurHC.save_as_text('newdictionary_JodhpurHC.txt') 
# dictionary_KolkataHC.save_as_text('newdictionary_KolkataHC.txt') 
# dictionary_SupremeCourt.save_as_text('newdictionary_SupremeCourt.txt') 
# Save actual dictionaries
dictionary_DCDRC.save('newdictionary_DCDRC') 
dictionary_NCDRC.save('newdictionary_NCDRC') 
dictionary_SCDRC.save('newdictionary_SCDRC') 
dictionary_DelhiHC.save('newdictionary_DelhiHC') 
dictionary_JharkhandHC.save('newdictionary_JharkhandHC') 
dictionary_JodhpurHC.save('newdictionary_JodhpurHC') 
dictionary_KolkataHC.save('newdictionary_KolkataHC') 
dictionary_SupremeCourt.save('newdictionary_SupremeCourt')

2018-03-16 14:55:10,891 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-16 14:56:54,091 : INFO : adding document #10000 to Dictionary(190225 unique tokens: [u'gudadannavar', u'coolege', u'withoutcomplying', u'ofaffidavitcumreply', u'metersand']...)
2018-03-16 14:58:25,245 : INFO : adding document #20000 to Dictionary(341648 unique tokens: [u'gudadannavar', u'srimohanchandra', u'coolege', u'withoutcomplying', u'ofday']...)
2018-03-16 14:59:52,238 : INFO : adding document #30000 to Dictionary(515380 unique tokens: [u'gudadannavar', u'gavecredit', u'mrmvenkatesancounsel', u'withoutcomplying', u'rkgejggll']...)
2018-03-16 15:01:07,052 : INFO : adding document #40000 to Dictionary(713926 unique tokens: [u'vaishnavaccountant', u'tdrwas', u'withoutcomplying', u'nexusvisvis', u'boncer']...)
2018-03-16 15:02:07,567 : INFO : adding document #50000 to Dictionary(840531 unique tokens: [u'vaishnavaccountant', u'tdrwas', u'withoutcomplying', u'nexusvisvis', u'boncer']...)
2018

2018-03-16 15:24:55,855 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-16 15:25:36,639 : INFO : adding document #10000 to Dictionary(179819 unique tokens: [u'horses', u'withoutcomplying', u'whichincites', u'otheractivitesas', u'metersand']...)
2018-03-16 15:26:18,284 : INFO : adding document #20000 to Dictionary(273204 unique tokens: [u'berma', u'horses', u'withoutcomplying', u'suchdefinition', u'whichincites']...)
2018-03-16 15:27:08,006 : INFO : adding document #30000 to Dictionary(368342 unique tokens: [u'purposenarendra', u'withoutcomplying', u'whichincites', u'arithmeticallyincorrect', u'brokenetowel']...)
2018-03-16 15:27:20,954 : INFO : built Dictionary(386629 unique tokens: [u'purposenarendra', u'withoutcomplying', u'whichincites', u'arithmeticallyincorrect', u'brokenetowel']...) from 32240 documents (total 26325476 corpus positions)
2018-03-16 15:27:20,988 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-16 15:27:41,015 : INFO : bu

In [7]:
# load those dictionaries
dictionary_DCDRC = corpora.Dictionary.load('newdictionary_DCDRC') 
dictionary_NCDRC = corpora.Dictionary.load('newdictionary_NCDRC') 
dictionary_SCDRC = corpora.Dictionary.load('newdictionary_SCDRC') 
dictionary_DelhiHC = corpora.Dictionary.load('newdictionary_DelhiHC') 
dictionary_JharkhandHC = corpora.Dictionary.load('newdictionary_JharkhandHC') 
dictionary_JodhpurHC = corpora.Dictionary.load('newdictionary_JodhpurHC') 
dictionary_KolkataHC = corpora.Dictionary.load('newdictionary_KolkataHC') 
dictionary_SupremeCourt= corpora.Dictionary.load('newdictionary_SupremeCourt')

2018-03-16 15:36:02,888 : INFO : loading Dictionary object from newdictionary_DCDRC
2018-03-16 15:36:03,849 : INFO : loaded newdictionary_DCDRC
2018-03-16 15:36:04,098 : INFO : loading Dictionary object from newdictionary_NCDRC
2018-03-16 15:36:04,182 : INFO : loaded newdictionary_NCDRC
2018-03-16 15:36:04,208 : INFO : loading Dictionary object from newdictionary_SCDRC
2018-03-16 15:36:04,528 : INFO : loaded newdictionary_SCDRC
2018-03-16 15:36:04,644 : INFO : loading Dictionary object from newdictionary_DelhiHC
2018-03-16 15:36:04,751 : INFO : loaded newdictionary_DelhiHC
2018-03-16 15:36:04,790 : INFO : loading Dictionary object from newdictionary_JharkhandHC
2018-03-16 15:36:04,918 : INFO : loaded newdictionary_JharkhandHC
2018-03-16 15:36:04,960 : INFO : loading Dictionary object from newdictionary_JodhpurHC
2018-03-16 15:36:05,100 : INFO : loaded newdictionary_JodhpurHC
2018-03-16 15:36:05,157 : INFO : loading Dictionary object from newdictionary_KolkataHC
2018-03-16 15:36:05,175 

In [8]:
# discard words occuring in less than 5 documents and in more than 50% 
dictionary_DCDRC.filter_extremes(keep_n=None)
dictionary_NCDRC.filter_extremes(keep_n=None) 
dictionary_SCDRC .filter_extremes(keep_n=None)
dictionary_DelhiHC.filter_extremes(keep_n=None) 
dictionary_JharkhandHC.filter_extremes(keep_n=None) 
dictionary_JodhpurHC.filter_extremes(keep_n=None) 
dictionary_KolkataHC.filter_extremes(keep_n=None) 
dictionary_SupremeCourt.filter_extremes(keep_n=None) 
# Merge all the dictionaries
dictionary = dictionary_DCDRC
dictionary.merge_with(dictionary_DelhiHC)
dictionary.merge_with(dictionary_NCDRC)
dictionary.merge_with(dictionary_SCDRC)
dictionary.merge_with(dictionary_SupremeCourt)
dictionary.merge_with(dictionary_JharkhandHC)
dictionary.merge_with(dictionary_JodhpurHC)
dictionary.merge_with(dictionary_KolkataHC)
dictionary.compactify()


2018-03-16 15:44:47,562 : INFO : discarding 1370740 tokens: [(u'all', 70971), (u'chempazhanthi', 3), (u'issued', 78775), (u'to', 120964), (u'beenakumariamember', 2), (u'th', 63546), (u'under', 94240), (u'narendranthis', 1), (u'cost', 69475), (u'further', 71982)]...
2018-03-16 15:44:47,563 : INFO : keeping 244791 tokens which were in no less than 5 and no more than 61447 (=50.0%) documents
2018-03-16 15:44:48,446 : INFO : resulting dictionary: Dictionary(244791 unique tokens: [u'filingoriginal', u'withoutcomplying', u'nonehru', u'partiesnilexhibits', u'distributers']...)
2018-03-16 15:44:49,042 : INFO : discarding 181434 tokens: [(u'ccomplaints', 2), (u'docdocnoconsumercourtncdrcdocnotextnational', 2487), (u'to', 3481), (u'th', 2030), (u'under', 2881), (u'ofthe', 2456), (u'theexpansion', 2), (u'cost', 1925), (u'further', 2307), (u'deducteda', 2)]...
2018-03-16 15:44:49,042 : INFO : keeping 27343 tokens which were in no less than 5 and no more than 1742 (=50.0%) documents
2018-03-16 15:4

In [9]:
d = copy.deepcopy(dictionary)

In [13]:
dictionary = copy.deepcopy(d)

In [10]:
len(d)

370358

In [16]:
len(dictionary)

76125

In [14]:
dictionary.filter_extremes(no_below=50)
# Need this after merging
dictionary.compactify()

2018-03-16 15:49:17,247 : INFO : discarding 294233 tokens: [(u'metersand', 9), (u'sowell', 6), (u'documentupon', 5), (u'trawling', 9), (u'ofjmic', 6), (u'canes', 25), (u'andprinciple', 7), (u'halfshare', 33), (u'canel', 6), (u'fkhbl', 11)]...
2018-03-16 15:49:17,248 : INFO : keeping 76125 tokens which were in no less than 50 and no more than 175992 (=50.0%) documents
2018-03-16 15:49:17,498 : INFO : resulting dictionary: Dictionary(76125 unique tokens: [u'withdamages', u'majumder', u'withoutcomplying', u'struck', u'aloksinghjmanishtextdoc']...)


In [15]:
# save the dictionary
dictionary.save('newdictionary')

2018-03-16 15:57:03,777 : INFO : saving Dictionary object under newdictionary, separately None
2018-03-16 15:57:03,918 : INFO : saved newdictionary


In [ ]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('newdictionary')

In [ ]:
len(dictionary.items())

In [ ]:
# This is BOW wrapper
c = my_corpus()
corpus = c.everything()
# save BOW
corpora.MmCorpus.serialize('newcorpus.mm', corpus)

In [ ]:
######################################## load BOW
corpus = corpora.MmCorpus('newcorpus.mm')

In [ ]:
# Tfidf model
# Will take time as to generate a model it processes over the whole corpus
tfidf = gensim.models.TfidfModel(corpus)
# save tfidf model
tfidf.save('newtfidf.mdl')

In [ ]:
##################################### load tfidf model
tfidf = gensim.models.tfidfmodel.TfidfModel.load('newtfidf.mdl')

In [ ]:
# tfidf wrapper coprus
# doesn't take time - yield
tfidf_corpus = tfidf[corpus]

# actual tfidf corpus serialized
corpora.MmCorpus.serialize('newtfidf_corpus.mm', tfidf_corpus)


In [ ]:
##################################### load tfidf corpus
tfidf_corpus = corpora.MmCorpus('newtfidf_corpus.mm')

In [ ]:
# get the query truth in dictionary
# query no. gives the list of 0-1 documents
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [ ]:
# create index - Tfidf
tfidf_index = gensim.similarities.Similarity('tfidf.index',tfidf_corpus,len(dictionary))

# save index - tfidf
gensim.similarities.Similarity.save(tfidf_index,'TFIDF_index')


In [ ]:
########################### load tfidf index
tfidf_index = gensim.similarities.Similarity.load('TFIDF_index')

In [ ]:
# calculate similarity for all 
tfidf_index.num_best= None

In [ ]:
#######################         SIMILIRATIES SCORES AND RANKING    ###########################
tfidf_results = copy.deepcopy(query_truth)
### lists of list (indexing from 1, dummy element)
sim_list = [0]
rank_list = [0]
queries = [0]
for i,query in enumerate(files_to_tokens('LegalAdhocTask/q*.txt')):
    queries.append(query)
    # inside the square bracket determines query representation
    
    sims = tfidf_index[tfidf[dictionary.doc2bow(query)]]
    sim_list.append(sims)
    # rank of every document wrt similarity
    ranks = rankdata(sims, method='ordinal')
    ranks= len(ranks)+1 - ranks 
    rank_list.append(ranks)
    
    # update the query truth tuples with similarity score and the ranks
    for x in tfidf_results[str(i+1)]:
        x.append(sims[filenames.index(x[0]+'.txt')])
        x.append(ranks[filenames.index(x[0]+'.txt')])
        #x.append(common_words(filenames.index(x[0]+'.txt'), tfidf_corpus, query))
    
    # sort wrt relevance(from truth) and then ranks(from our model)
    #tfidf_results[str(i+1)].sort(key=lambda x: (-int(x[1]),x[3]))     
    
        

In [ ]:
tfidf_results


In [ ]:
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook

In [ ]:
# Store results in 1st sheet
wb = Workbook()
dest_filename = 'tfidf.xlsx'
ws1 = wb.active
ws1.title = "TFIDF"
ws1.append(['Query','Filename', 'Relevance', 'Score', 'Rank'])
for key,value in tfidf_results.iteritems():
    for i in value:
        ws1.append([int(key)]+ i)

wb.save(filename = dest_filename)

In [ ]:
#evaluation_Qwise - Precision, Recall and Fscore

import pandas as pd

f = pd.ExcelFile('tfidf.xlsx')

from scipy.stats import hmean



model_name = 'tfidf'
places = 4
wb = load_workbook('tfidf.xlsx')
ws1 = wb.create_sheet(title="evaluation_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])

k = [1.0, 5.0 ,10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]
for sheet in f.sheet_names:
    if(sheet == "evaluation"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total_1 = (float)(x[x['Relevance'] == 1].shape[0])
        total_0 = (float)(x[x['Relevance'] == 0].shape[0])
        total = (float)(x.shape[0])
        
        ####  repitition due to complication in considering both 1 & 0 relevance
        # precision 1
        row = [q, 1, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/i)
        ws1.append(row);
        p1 = row;
        # precision 0
        row = [q, 0, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/i)    
        ws1.append(row);
        p0 = row;
        #precision 10
        row = [q, 10, 'P', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/i)  
        ws1.append(row);
        p10 = row;  
        
        
        # recall 1
        row = [q, 1, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 1 , x['Rank'] <= i)].shape[0]/total_1)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r1 = row;    
        # recall 0
        row = [q, 0, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[np.logical_and(x['Relevance'] == 0 , x['Rank'] <= i)].shape[0]/total_0)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r0 = row;
        #recall 10
        row = [q, 10, 'R', model_name + ' :' + sheet]
        for i in k:
            row.append(x[x['Rank'] <= i].shape[0]/total)
            row[-1] = round(row[-1],places)
        ws1.append(row);
        r10 = row;      


        # F 1
        row = [q, 1, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p1[i] == 0.0 :
                row.append(0.0)
            else:
                row.append(hmean([p1[i],r1[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 0
        row = [q, 0, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p0[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p0[i],r0[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);

        # F 10
        row = [q, 10, 'F', model_name + ' :' + sheet]
        for i in range(-len(k), 0, 1):
            if p10[i] == 0.0:
                row.append(0.0)
            else:
                row.append(hmean([p10[i],r10[i]]))
            row[-1] = round(row[-1],places)
        ws1.append(row);
        
wb.save(filename = 'tfidf.xlsx')

In [ ]:
#Query wise Average precision
# precision sum
def p_sum(z):
    z = z.copy()  
    z.sort_values(inplace=True)
    result = 0
    for i,val in enumerate(z):
        result += (i+1)/float(val) 
    return result


import pandas as pd

f = pd.ExcelFile('tfidf.xlsx')

from scipy.stats import hmean


model_name = 'tfidf'
places = 4
wb = load_workbook('tfidf.xlsx')
ws1 = wb.create_sheet(title="AP_Qwise")

ws1.append(['Query','Docs considered', 'Precision/Recall', 'Model', 1, 5 ,10, 25, 50, 100, 500, 1000])


for sheet in f.sheet_names:
    
    if(sheet == "evaluation"or sheet == "evaluation_Qwise"):
        continue
    for q in np.arange(1,11):

        x = f.parse(sheet)
        x = x[x['Query'] == q]
        total = {}
        total[1] = (float)(x[x['Relevance'] == 1].shape[0])
        total[0] = (float)(x[x['Relevance'] == 0].shape[0])
        total[10] = (float)(x.shape[0])
        
        for rel in [1,0]:
            # precision 1
            row = [q, rel, 'AP', model_name + ' :' + sheet]
            for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
                row.append(p_sum(x[np.logical_and(x['Relevance'] == rel, x['Rank'] <= i)]['Rank'])/total[rel])
            ws1.append(row);

        row = [q, 10, 'AP', model_name + ' :' + sheet]
        for i in [1.0, 5.0, 10.0, 25.0, 50.0, 100.0, 500.0, 1000.0]:
            row.append(p_sum(x[x['Rank'] <= i]['Rank'])/total[10])
        ws1.append(row);
wb.save(filename = 'tfidf.xlsx')